In [13]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GroupKFold

In [3]:
df_train = pd.read_csv('train_with_features.csv')

## TODO: fill with 0 or median depending on the column

In [4]:
df_train = df_train.fillna(0)

In [14]:
kfold = GroupKFold()


In [6]:
df_train.stock_id = df_train.stock_id.astype('category')

In [7]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [8]:
cat_nn.remove('time_id'), cat_nn.remove('row_id')

(None, None)

In [16]:
def train_fold(trn_idx, val_idx, fname):
    

    procs_nn = [Categorify, Normalize]
    to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                          splits=[list(trn_idx), list(val_idx)], y_names='target')

    dls = to_nn.dataloaders(1024)
    learn = tabular_learner(dls, y_range=(0,.1), layers=[1000,500,200],config={'lin_first':False},
                        n_out=1, loss_func=rmspe_loss, metrics=AccumMetric(rmspe),
                        #cbs=[EarlyStoppingCallback(monitor='rmspe', comp=np.less, patience=20),
                        #    SaveModelCallback(monitor='rmspe', comp=np.less, fname=fname)]
                           )
    learn.fit_one_cycle(70, 5e-3)
    learn.save(fname)
    return learn.recorder.metrics[0].value

In [10]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [11]:
def rmspe_loss(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [17]:
res =[]
for trn_idx, val_idx in kfold.split(df_train, groups = df_train.time_id):
    res.append(train_fold(trn_idx, val_idx, f'kfold_{len(res)}'))
    print('fold', len(res), res[-1])
np.mean(res),res

epoch,train_loss,valid_loss,rmspe,time
0,0.253413,0.241553,0.247580,00:02
1,0.238790,0.240037,0.242641,00:02
2,0.236459,0.235351,0.238349,00:03
3,0.240799,0.255531,0.257110,00:02
4,0.240993,0.236239,0.239046,00:02
5,0.261667,0.229154,0.231511,00:02
6,0.241814,0.236244,0.238999,00:02
7,0.240134,0.227775,0.230134,00:02
8,0.237480,0.229593,0.232060,00:02
9,0.240556,0.237034,0.239049,00:02


fold 1 TensorBase(0.2427)


epoch,train_loss,valid_loss,rmspe,time
0,0.251400,0.237692,0.240158,00:04
1,0.243022,0.247629,0.252314,00:04
2,0.246091,0.261528,0.264180,00:04
3,0.243360,0.257632,0.259759,00:03
4,0.233052,0.239733,0.243325,00:04
5,0.234295,0.234716,0.236839,00:03
6,0.236461,0.239644,0.241859,00:03
7,0.242854,0.232612,0.234826,00:03
8,0.236047,0.234243,0.239814,00:04
9,0.234163,0.232227,0.234660,00:03


fold 2 TensorBase(0.2570)


epoch,train_loss,valid_loss,rmspe,time
0,0.260872,0.242854,0.245385,00:03
1,0.241937,0.233630,0.235914,00:03
2,0.239594,0.229080,0.231258,00:03
3,0.243702,0.225973,0.229483,00:03
4,0.241008,0.225474,0.227143,00:03
5,0.241951,0.224714,0.226927,00:04
6,0.238237,0.229537,0.231432,00:03
7,0.239172,0.224019,0.226836,00:04
8,0.240533,0.226215,0.228647,00:03
9,0.240124,0.219986,0.221916,00:04


...

epoch,train_loss,valid_loss,rmspe,time
0,0.256650,0.230840,0.236180,00:04
1,0.245059,0.222664,0.225251,00:03
2,0.245181,0.231983,0.239715,00:04
3,0.243415,0.245270,0.250437,00:03
4,0.244282,0.233025,0.235071,00:03
5,0.245207,0.242361,0.244074,00:03
6,0.236860,0.220651,0.222907,00:03
7,0.239496,0.240629,0.243475,00:03
8,0.239187,0.221389,0.223256,00:03
9,0.237595,0.250715,0.252560,00:03


fold 4 TensorBase(0.2448)


epoch,train_loss,valid_loss,rmspe,time
0,0.252232,0.228151,0.229945,00:03
1,0.247163,0.223380,0.227288,00:04
2,0.241264,0.231776,0.233547,00:05
3,0.244206,0.224011,0.225898,00:03
4,0.243321,0.222781,0.224782,00:04
5,0.247167,0.225277,0.227212,00:03
6,0.250760,0.230232,0.233178,00:03
7,0.246538,0.227311,0.228683,00:03
8,0.234706,0.216283,0.218036,00:03
9,0.240421,0.232620,0.236103,00:04


fold 5 TensorBase(0.2379)


(0.2447594,
 [TensorBase(0.2427),
  TensorBase(0.2570),
  TensorBase(0.2414),
  TensorBase(0.2448),
  TensorBase(0.2379)])